# Ray framework

## Obsah
- [Úvod](#1)
- [Obecně o Ray framework](#2)
- [Cluster - obecně](#3)
- [Ray Cluster](#4)
    - [Head node](#5)
    - [Worker nodes](#6)
- [Ray framework z pohledu vývojáře Python aplikací](#7)
    - [Ray Core](#8)
    - [Připojení k existujícímu clusteru](#9)
    - [Poznámka k Ray na Windows](#10)
    - [Vzdálený cluster](#11)
- [Ray Framework z pohledu infrastruktury](#12)
- [Praktická ukázka](#13)
    - [Tvorba clusteru](#14)
        - [Příprava infrastruktury](#15)
        - [Instalace Ray a spuštění clusteru](#16)
        - [Řešené problémy](#17)
            - [Vzdálený vývoj](#18)
            - [Přidání Ray do PATH](#19)
    - [AI](#20)
        - [Úprava kódu pro distribuovanou AI](#21)
    - [Trénink AI v clusteru](#22)
- [Závěr](#23)
- [Zdroje](#24)

<a id="1"></a>
# Úvod
- cílem tohoto notebooku je seznámit s praktickým nasazením distribuované AI
- k dosažení tohoto cíle je potřeba pochopit základní principy fungování poskytovaného frameworku

- tento notebook bych rád rozdělil na tři části:
    1. [Ray framework z pohledu vývojáře Python aplikací](#7)
        - tato část bude sloužit pro pochopení změn v kódu aplikace (AI), které byly nezbytné, aby AI mohla být distibuovaná
        - tzn. mám k dispozici cluster a starám se jen o psaní kódu, využívám služeb clusteru
    2. [Ray Framework z pohledu infrastruktury](#12)
        - tato část bude sloužit pro získání dovedností nutných k vybudování vlasního malého clusteru
        - tzn. chci vybudovat a spravovat svůj cluster a poskytovat ho pro vývojáře
    3. [Praktická ukázka](#13)
        - nasazení distribuované AI na vytvořený cluster

<a id="2"></a>
# Obecně o Ray Framework
- jedná se o nástroj který slouží ke škálování aplikací
- hlavní myšlenkou je možnost škálování aplikace "from laptop to datacentre with little to no code changes"
- postupně se rozrostl na několik součástí:
    - Ray Core
    - Ray Clusters
    - Ray Data
    - Ray Train
    - Ray Tune
    - Ray Serve
    - Ray RLlib

<a id="3"></a>
# Cluster - obecně
- computer cluster is a set of computers (nodes) that work together so that they can be viewed as a single system __[(zdroj)](https://en.wikipedia.org/wiki/Computer_cluster)__
- obvykle se skládá z control node a compute nodes
- control node řídí celý cluster, přiděluje úlohy a stará se o komunikaci s uživatelem
- compute nodes vykonávají výpočty a výsledky předávají do control node
- z pohledu uživatele clusteru tak předáváme úlohu k výpočtu jednomu počítači (control node) - nestaráme se o vnitřní fungování clusteru (distribuci na compute nodes)

<a id="4"></a>
# Ray Cluster
- Ray framework pracuje s výše popsaným konceptem clusteru, jen používá mírně jinou terminologii pro nodes
- rozlišuje head node (=control node) a worker node (=compute node)

![Ray Cluster](pictures\cluster.png)

- počet worker nodů lze dynamicky měnit podle zátěže - tzn. lze provádět autoscaling
- cluster bývá v praxi velmi výkonný server nebo skupina serverů, popř. VM na cloudu - Ray podporuje AWS a Azure
- v rámci jednoho node funguje paralelizace na úrovni vláken - i na jednom nodu mohou běžet dvě a více úloh současně

<a id="5"></a>
## Head node
- každý cluster musí obsahovat head node
- pokud spustíme Ray jen na našem zařízení (např. notebook), tak de facto vytvoříme cluster o jednom head nodu a žádných worker nodes
- head node je téměř stejný jako worker node, jen na něm navíc běží další procesy pro obsluhu clusteru
- je to jediný node, se kterým budeme jako uživatelé clusteru komunikovat

<a id="6"></a>
## Worker nodes
- slouží pouze ke spouštění kódu
- sdílí mezi sebou cluster memory

<a id="7"></a>
# 1. Ray framework z pohledu vývojáře Python aplikací
- v této části nás bude nejvíce zajímat Ray Core a práce s API tzn. zajímáme se o to, jak psát programy pro Ray clustery

<a id="8"></a>
# Ray Core
- poskytuje základní entity, se kterými Ray pracuje - jedná se o tasks (odpovídá python funkcím), actors (odpovídá python instancím), objects (odpovídá python objektům)
- je to původní a první modul celého frameworku
- poskytuje možnost spuštění nejmenšího "clusteru" - clusterem je v tomto případě náš počítač na kterém běží head node
- i v takto minimální konfiguraci lze těžit z výhod Ray Core - umí paralelizovat výpočty na jendom stroji - v rámci nodu běží více procesů
- lze se tak odstínit od paralelizace a nechat vše na frameworku

In [ ]:
pip install -U "ray[default]" #instalace Ray frameworku - tato konfigurace umožňuje jeho využití s jakoukoliv apliakací - ne jen AI

- komunikaci s __lokálním__ clusterem provádíme pomocí Ray API
- stačí nám volat funkce z knihovny ray
- framework se postará o spuštění clusteru a odeslání úlohy na něj
- Python funkce popř. objekty určené pro zpracování v clusteru se dekorují pomocí @ray.remote
- při volání fcí se opět používá .remote()
- jakmile funkci zavoláme, tak je odeslána ke zpracování do clusteru (zde se jedná jen o náš ntb = head node)
- kód samotné aplikace pokračuje i po zavolání fce - nečeká se na její dokončení - výpočet dekorované remote fce se provádí v clusteru (v praxi by např. běžel v cloudu)
- funkce tedy nevrací hodnotu, ale vrací pouze příslib budoucího výsledku
- pokud chceme v aplikaci počkat na dokončení výpočtu, musíme zavolat fci ray.get() do které jako parametr vložíme future promise

In [1]:
import ray
import time

@ray.remote #zde je možné specifikovat nějaké parametry spuštění např. @ray.remote(num_cpus=4)
def square(x):
    return x * x

ray.init() # zde provedeme spuštění local head node - na něj budeme odesílat úlohy
#start = time.time()

futures = [] # připravený list na future results (= promises)

for i in range(100):
    futures.append(square.remote(i)) #futures je list příslibů - odkaz na budoucí výsledek - konkrétní výsledek dostaneme po zavolání fce ray.get()
    # ray.get() lze volat na celý list příslibů - viz dole ray.get(futures) nebo jen na jeden záznam např. ray.get(futures[10])
    # po zavolání get se čeká na dokončení výsledku výpočtu


print("Pokracuji si klidně dál") #program zde může pokračovat - výpočet běží v clusteru - na výsledky si počkám až když zavolám fci get
data = ray.get(futures) #data je klasický list výsledků [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

print(data)
print(type(data))

#print(time.time()-start)

ray.shutdown() #ukončí cluster - ukončí se po skončení programu, zde ale nutné - problém v jupyter kernelu

c:\Users\remar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-16 12:47:11,925	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-16 12:47:15,034	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Pokracuji si klidně dál
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]
<class 'list'>


- obdobně lze v clusteru definovat i objekty - ty se v ray core definují jako actors - lze mít na clusteru objekt a pracovat s ním

In [1]:
import ray

@ray.remote
class Counter:
    def __init__(self):
        self.i = 0

    def get_data(self):
        return self.i
    
    def incr(self, value):
        self.i += value


ray.init()

c = Counter.remote() #jedná se pouze o actor handle - nejedná se o klasický python objekt - objekt se vytvoří na daném clusteru
# pomocí actor handle se můžeme odkazovat na tuto instanci v clusteru
print(type(c))

for _ in range(10):
    c.incr.remote(10) #voláme fci na instanci v clusteru

print(ray.get(c.get_data.remote())) #počkáme si na výsledek fce a vypíšeme

ray.shutdown()

c:\Users\remar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-16 12:48:07,156	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-16 12:48:10,229	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


<class 'ray.actor.ActorHandle'>
100


Jak je patrné z ukázek, tak použití ray core je velmi jednoduché a opravdu lze velmi snadno stávající kód upravit tak, aby byl připraven pro nasazení v clusteru.
Výpočetně náročné operace tak lze snadno přenést na výkonný stroj.

In [1]:
import ray
import time

@ray.remote
class MessageActor: # actor určený pro sběr zpráv
    def __init__(self):
        self.messages = [] # list pro uložení zpráv

    def add_message(self, message):
        self.messages.append(message)

    def get_and_clear(self):
        out = self.messages
        self.messages = []
        return out
    
@ray.remote
def worker(message_actor, worker_no): # funkce worker generuje zprávy a předává je do objektu message actor
    for i in range(20):
        time.sleep(1)
        message_actor.add_message.remote(f"Zprava od {worker_no} cislo {i}")

message_actor = MessageActor.remote()

for worker_no in range(3):
    worker.remote(message_actor, worker_no)

for _ in range(100):
    new_messages = ray.get(message_actor.get_and_clear.remote())
    print(f"Nove zpravy: {new_messages}")
    time.sleep(1)

ray.shutdown()

c:\Users\remar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-17 12:59:12,499	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-17 12:59:17,846	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Nove zpravy: []
Nove zpravy: ['Zprava od 1 cislo 0', 'Zprava od 0 cislo 0', 'Zprava od 2 cislo 0']
Nove zpravy: ['Zprava od 2 cislo 1', 'Zprava od 0 cislo 1', 'Zprava od 1 cislo 1']
Nove zpravy: ['Zprava od 2 cislo 2', 'Zprava od 1 cislo 2', 'Zprava od 0 cislo 2']
Nove zpravy: ['Zprava od 1 cislo 3', 'Zprava od 2 cislo 3', 'Zprava od 0 cislo 3']
Nove zpravy: ['Zprava od 0 cislo 4', 'Zprava od 1 cislo 4', 'Zprava od 2 cislo 4']
Nove zpravy: ['Zprava od 0 cislo 5', 'Zprava od 2 cislo 5', 'Zprava od 1 cislo 5']
Nove zpravy: ['Zprava od 2 cislo 6', 'Zprava od 1 cislo 6', 'Zprava od 0 cislo 6']
Nove zpravy: ['Zprava od 0 cislo 7', 'Zprava od 1 cislo 7', 'Zprava od 2 cislo 7']
Nove zpravy: ['Zprava od 1 cislo 8', 'Zprava od 0 cislo 8', 'Zprava od 2 cislo 8']
Nove zpravy: ['Zprava od 1 cislo 9', 'Zprava od 0 cislo 9', 'Zprava od 2 cislo 9']
Nove zpravy: ['Zprava od 2 cislo 10', 'Zprava od 0 cislo 10', 'Zprava od 1 cislo 10']
Nove zpravy: ['Zprava od 0 cislo 11', 'Zprava od 1 cislo 11', 'Zprav

KeyboardInterrupt: 

<a id="9"></a>
## Připojení k existujícímu clusteru
- do teď jsme používali funkci ray.init(), která při volání bez parametrů nejprve hledá již existující cluster a pokud ho nenajde, tak si vytvoří nový
- obvykle budeme používat již spuštěný a existující cluster
- pro studijní účely si můžeme toto simulovat pomocí spuštění head node z příkazové řádky - head node poběží pořád a bude jen čekat na úlohy
- pro spuštění a správu ray nodes na našem stroji je možné použít tyto příkazy (stejné pro Win/Linux):
    - ray start --head 
    - ray status
    - ray stop

<a id="10"></a>
## Poznámka k Ray na Windows
- v době psaní tohoto notebooku Ray cluster plně nepodporuje Windows - pod tímto OS lze pouze spustit head node - nelze přiřadit Windows stanici jako worker do již existujícího clusteru a ani není možné k head node běžícím pod Windows připojit jakékoliv další worker nodes
- pro studijní/testovací účely je ale head node dostatečný a lze si na něm vyzkoušet práci s Ray frameworkem

## Prakticky
Windows cmd:

In [ ]:
ray start -- head

In [ ]:
ray.init(address="auto") # provede automatickou detekci běžících clusterů - pokud žádný nenajde -> error

- takto spuštěný cluster nám na localhost:8265 poskytuje dashboard
- pokud head node neběží pod Windows, tak k němu lze připojovat worker nodes a budovat tak cluster

<a id="11"></a>
## Vzdálený cluster
- pokud pracujeme na stroji, který není součástí clusteru - lze pomocí Ray spustit vzdáleně úlohu

In [ ]:
from ray.job_submission import JobSubmissionClient

client = JobSubmissionClient("http://192.168.88.240:8265")
job_id = client.submit_job(
    # Entrypoint shell command to execute
    entrypoint="python3 /home/user/Ray_test/task.py"
    # Path to the local directory that contains the script.py file
)
print(job_id)

<a id="12"></a>
# 2. Ray Framework z pohledu infrastruktury
- vlastní Ray cluster lze vytvořit na:
    - Kubernetes
    - AWS, GCP, Azure, vSphere
    - Virtual machines
- nejjednodušší a uživatelsky nejlepší mi přijde vytvářet clutery v cloudu - výborné propojení s AWS - clustery pak lze jednoduše definovat pomocí .yaml - viz. __[zde](https://docs.ray.io/en/latest/cluster/vms/references/ray-cluster-configuration.html?highlight=yaml)__

- protože nemám přístup k cloudovým zdrojům, tak jsem se rozhodl vyzkoušet si nasazení Ray na VM

<a id="13"></a>
# 3. Praktická ukázka
- cílem je prakticky předvést distribuovanou AI na vlastním clusteru, který bude vytvořen pomocí virtuálních počítačů

<a id="14"></a>
## Tvorba clusteru
- rozhodl jsem se vyvtvořit si malý virtuální cluster o jednom head node a jednom worker node
- z důvodu nekompatibility Ray s Windows bylo butné si vytvořit dva virtuální servery s Ubuntu server

<a id="15"></a>
### Příprava infrastruktury
- rozhodl jsem se použít virtulizační nástroj virtualbox, ve kterém jsem si do VM nainstaloval Ubuntu 22.04 server
- do VM jsem nainstaloval Python 3.10 a pip
- takto připravenou VM jsem si naklonoval
- u klonu jsem změnil MAC adresu jeho adaptéru, hostname a ip adresu
- oba servery jsem dal na stejnou virtuální síť spolu s host PC
- vznikla tak tato topologie:
![Topologie](pictures\topologie.png)

<a id="16"></a>
### Instalace Ray a spuštění clusteru
 - na obou strojích jsem provedl instalaci Ray frameworku - jak default verzi pro obecný vývoj, tak i rozšířené knihovny pro AI

In [ ]:
pip install -U "ray[default]"
pip install -U "ray[data,train,tune,serve]"

- Nyní je možné spustit cluster
- začneme s head node, který spustíme pomocí příkazu

- přidáním parametru --dashboard-host 0.0.0.0 zpřístupníme dashboard na všech IP adresách, které server má (ne jen localhost) - můžeme si dashboard zobrazit z host pc
- worker node spustíme a připojíme na head node pomocí příkazu

- takto spuštěný cluster nám na  192.168.88.240:8265 poskytuje dashboard, kde v záložce cluster můžeme vidět, že se nám podařilo vytvořit cluster o dvou nodech
- pokud tedy nyní na počítači raynode spustíme některý z předchozích ukázkových kódu, tak se výpočet provede na clusteru - bude ho provádět i worker node

<a id="17"></a>
### Řešené problémy

<a id="18"></a>
Vzdálený vývoj
- v této fázi projektu jsem se dostal do situace, kdy jsem chtěl s clusterem nějak pohodlně pracovat
- chtěl jsem docílit toho, že budu vyvíjet aplikaci na svém pc (v tomto případě na host pc) a úlohy se budou provádět v clusteru
- díky tomu, že používám Windows, tak nebylo možné můj host pc přidat do clusteru - nedařilo se mi pohodlně spouštět úlohy
- zkoušel jsem tedy různě "ohýbat" ray.init __[(docs)](https://docs.ray.io/en/latest/ray-core/api/doc/ray.init.html)__ tak, aby se mi podařilo odesílat úlohy na cluster rovnou z python kódu
- vyzkoušel jsem různá řešení, včetně zrcadlení portů mého pc do portů head nodu atd.
- žádné ze zkoušených řešení ale nepřineslo očekávaný efekt a pohodlí při vývoji
- jako jediné řešení se jevilo napsat aplikaci, přenést ji do head node a tam ji z command line spustit, popř. používat vzdálené spouštění kódu pomocí job submission __[(docs)](https://docs.ray.io/en/latest/cluster/running-applications/job-submission/doc/ray.job_submission.JobSubmissionClient.submit_job.html)__
- nakonec jsem se rozhodl použít modul remote-ssh, který lze nainstalovat do VS code - tento modul poskytuje možnost vzdáleného vývoje přes SSH - VS code se tváří, jako by aplikace běžela lokálně, ale ve skutečnosti běží na head node

<a id="19"></a>
Přidání Ray do PATH
- po instalaci se Ray framework nepřidal do PATH v systému Ubuntu
- bylo nutné spouštět ray pomocí úplné cesty, např:


- podle __[návodu](https://phoenixnap.com/kb/linux-add-to-path)__ bylo nutné přidat Ray do PATH

<a id="20"></a>
## AI
- jako ilustrační AI jsem vybral Iris AI, kterou jsem si půjčil __[zde](https://github.com/hrbolek/learning/blob/master/notebooks/ais/58_.ipynb)__
- AI jsem si nejprve upravil tak, abych mohl provést trénování modelu klasicky na svém stroji bez Ray framework
- jedná se o zcela běžné použití knihovny PyTorch:

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")


iris = datasets.load_iris()
def namedata(data):
    return {
        'sepal_l': data[0],
        'sepal_w': data[1],
        'petal_l': data[2],
        'petal_w': data[3]
    }
table = list(map(namedata, iris.data))

tmap = {0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}
targets = list(map(lambda item: tmap[item], iris.target))
table = [{**row, 'species': name} for row, name in zip(table, targets)]

df = pd.DataFrame(table)

le = preprocessing.LabelEncoder()

x = df[["sepal_l", "sepal_w", "petal_l", "petal_w"]].values
y = le.fit_transform(df["species"])

species = le.classes_

x = torch.tensor(x, device=device, dtype=torch.float32)
y = torch.tensor(y, device=device, dtype=torch.long)

model = nn.Sequential(
    nn.Linear(x.shape[1], 50),
    nn.ReLU(),
    nn.Linear(50, 25),
    nn.ReLU(),
    nn.Linear(25, len(species)),
    nn.LogSoftmax(dim=1),
)

model = torch.compile(model,backend="aot_eager").to(device)
criterion = nn.CrossEntropyLoss()  # cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(10000):
    optimizer.zero_grad()
    out = model(x)
    # Note: CrossEntropyLoss combines nn.LogSoftmax() and nn.NLLLoss() so don't use Softmax in the model
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, loss: {loss.item()}")

- počet epoch učení jsem uměle zvýšil - je to jen z důvodu zvýšení náročnosti procesu učení

<a id="21"></a>
## Úprava kódu pro distribuovanou AI
- nyní jsem podle __[dokumentace](https://docs.ray.io/en/latest/train/getting-started-pytorch.html)__ upravil AI tak, aby mohla fungoval s modulem Ray train:


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import ray
from sklearn import datasets
from sklearn import preprocessing
from ray.train.torch import TorchTrainer
from ray.train import ScalingConfig, Checkpoint, report
from ray.train.torch import prepare_model
from ray.train.torch import prepare_data_loader

def train_func(config):
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "cpu"
    )
    print(f"Using device: {device}")


    iris = datasets.load_iris()
    def namedata(data):
        return {
            'sepal_l': data[0],
            'sepal_w': data[1],
            'petal_l': data[2],
            'petal_w': data[3]
        }
    table = list(map(namedata, iris.data))

    tmap = {0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}
    targets = list(map(lambda item: tmap[item], iris.target))
    table = [{**row, 'species': name} for row, name in zip(table, targets)]

    df = pd.DataFrame(table)

    le = preprocessing.LabelEncoder()

    x = df[["sepal_l", "sepal_w", "petal_l", "petal_w"]].values
    y = le.fit_transform(df["species"])

    species = le.classes_

    x = torch.tensor(x, device=device, dtype=torch.float32)
    y = torch.tensor(y, device=device, dtype=torch.long)

    model = nn.Sequential(
        nn.Linear(x.shape[1], 50),
        nn.ReLU(),
        nn.Linear(50, 25),
        nn.ReLU(),
        nn.Linear(25, len(species)),
        nn.LogSoftmax(dim=1),
    )

    #model = torch.compile(model,backend="aot_eager").to(device)
    model = prepare_model(model)
    criterion = nn.CrossEntropyLoss()  # cross entropy loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    #train_loader = prepare_data_loader(train_loader)

    model.train()
    for epoch in range(1000):
        optimizer.zero_grad()
        out = model(x)
        # Note: CrossEntropyLoss combines nn.LogSoftmax() and nn.NLLLoss() so don't use Softmax in the model
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        report({"loss": loss.item()})
            
ray.init(address="auto")
scaling_config = ScalingConfig(num_workers=7, use_gpu=False) #zde si cluster žádám o resources - můj cluster má 8cpus - lze požádat o 7 workers (1 cpu je head)
trainer = TorchTrainer(train_func, scaling_config=scaling_config)
result = trainer.fit()

- jak je patrné z kódu pro distibuované učení AI, tak je možné definovat si pro danou úlohu resources - tzn. lze si cluster požádat o přidělení konkrétních zdrojů pro provedení výpočtu
- protože jsem virtualizaci clusteru prováděl na svém notebooku pomocí nástroje virtualbox, tak se mi nepodařilo spustit oba servery najednou tak, aby měly přístup k gpu - z toho důvodu jsem nechal výpočty běžet jen na cpu -> parametr use_gpu=False
- pro spuštění AI v mém clusteru bylo potřeba do obou serverů nainstalovat potřebné dependencies:

<a id="22"></a>
# Trénink AI v clusteru
- po spuštění úlohy na head node je možné během učení neuronové sítě sledovat zvýšenou zátěž obou serverů - tzn. došlo k distribuci učení AI na celý cluster - v našem případě na head a worker node
- takto natrénovaný model lze dále používat - lze ho nechat na clusteru a jen do něj posílat úlohy

<a id="23"></a>
# Závěr
Ray framework je silný nástroj pro paralelizaci úloh a tvorbu clusterů. Poskytuje mnoho dalších nástrojů a knihoven, které pokrývají celý životní cyklus vývoje AI - od zpracování dat přes trénink, ladění modelu až po poskytnutí modelu. Vývojář je tak zcela odstíněn od paralelizace výpočtů a framework podle mě skutečně naplňuje své odvážné marketingové tvrzení "from laptop to datacentre with little to no code changes".

Důkazem může být i použití Ray clusterů při učení neuronové sítě pro chat-gpt.

<a id="24"></a>
# Zdroje

- https://medium.com/juniper-team/tips-on-installing-and-maintaining-ray-cluster-b5535743f97c
- https://saturncloud.io/blog/getting-started-with-ray-clusters/
- https://docs.ray.io/en/latest/cluster/vms/references/ray-cluster-configuration.html?highlight=yaml
- https://docs.ray.io/en/latest/ray-core/api/doc/ray.init.html
- https://medium.com/@fengliplatform/ray-quick-hands-on-ecf744eb304f
- https://docs.ray.io/en/latest/train/getting-started-pytorch.html
- https://docs.ray.io/en/latest/cluster/running-applications/job-submission/sdk.html#ray-job-sdk
- https://docs.ray.io/en/latest/ray-overview/installation.html
- https://docs.ray.io/en/latest/cluster/running-applications/job-submission/doc/ray.job_submission.JobSubmissionClient.submit_job.html